## Build Your First Intelligent Agent Team: A Research Asisstant Tool for Curing Diabetes with ADK

https://google.github.io/adk-docs/get-started/quickstart/

https://google.github.io/adk-docs/tutorials/agent-team/ 

### Set up Environment & Install ADK

In [1]:
!pip install google-adk


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


### Import necessary libraries

In [2]:
# @title Import necessary libraries
import os
import asyncio
from google.adk.agents import Agent
from google.adk.agents import LlmAgent
#from google.adk.models.lite_llm import LiteLlm # For multi-model support
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types # For creating message Content/Parts

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.ERROR)

print("Libraries imported.")

Libraries imported.


#### Configure API Keys (Replace with your actual keys!)

In [3]:
from dotenv import load_dotenv
import os

In [4]:
load_dotenv()  # This loads variables from .env into the environment


True

In [5]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [6]:
import os

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY


In [7]:
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [8]:
# --- Define Model Constants for easier use ---

# More supported models can be referenced here: https://ai.google.dev/gemini-api/docs/models#model-variations
MODEL_GEMINI_2_0_FLASH = "gemini-2.0-flash"

print("\nEnvironment configured.")


Environment configured.


In [10]:
import sys
# Add the root directory of your project to Python path
project_root = os.path.abspath("../backend")
if project_root not in sys.path:
    sys.path.append(project_root)

In [54]:
path_to_credentials = os.getenv("PATH_TO_CREDENTIALS")

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = path_to_credentials


## Connect to the MongoDB collection

In [12]:
from db.mongodb_client import mongodb_client

✅ MongoDB connection established.


In [13]:
# Names of the MongoDB database, collection and vector search index
DB_NAME = "diabetes_data"
COLLECTION_NAME = "records_embeddings"
VS_INDEX_NAME = "csv_vector_index"

In [14]:
db = mongodb_client[DB_NAME]

In [15]:
# Connect to the MongoDB collection
collection = mongodb_client[DB_NAME][COLLECTION_NAME]

In [16]:

print("Client:", mongodb_client)
print("Databases:", mongodb_client.list_database_names())

Client: MongoClient(host=['ac-t38w2lp-shard-00-02.q4fmjuw.mongodb.net:27017', 'ac-t38w2lp-shard-00-01.q4fmjuw.mongodb.net:27017', 'ac-t38w2lp-shard-00-00.q4fmjuw.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='Search4Cure.diabetes', authsource='admin', replicaset='atlas-lunm1g-shard-0', tls=True, server_api=<pymongo.server_api.ServerApi object at 0x316210650>)
Databases: ['diabetes_data', 'admin', 'local']


In [17]:
import tiktoken

In [25]:
import time

In [23]:
import google.generativeai as genai

In [28]:
import tabulate

In [29]:
from pymongo.operations import SearchIndexModel

In [75]:
DB_NAME = os.getenv("MONGO_DB")
ATLAS_VECTOR_SEARCH_INDEX = "csv_vector_index"
CSV_COLLECTION= "records_embeddings"

In [76]:
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_mongodb.retrievers import MongoDBAtlasHybridSearchRetriever


In [77]:
from db.mongodb_client import MONGODB_URI

In [78]:

from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding_model = GoogleGenerativeAIEmbeddings(
    model=MODEL_GEMINI_2_0_FLASH,
    task_type="RETRIEVAL_DOCUMENT"
)

In [121]:
GEMINI_EMBEDDING_MODEL = "models/embedding-001"#"models/gemini-embedding-exp-03-07"

In [150]:

from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding_model = GoogleGenerativeAIEmbeddings(
    model=GEMINI_EMBEDDING_MODEL,
    task_type="RETRIEVAL_DOCUMENT"
)

In [151]:
vector_store_csv_files = MongoDBAtlasVectorSearch.from_connection_string(
    connection_string=MONGODB_URI,
    namespace=DB_NAME + "." + CSV_COLLECTION,
    embedding=embedding_model,
    index_name=ATLAS_VECTOR_SEARCH_INDEX,
    text_key="combined_info",
)

## Building an Agent Team 

In Steps 1 and 2, we built and experimented with a single agent focused solely on weather lookups. While effective for its specific task, real-world applications often involve handling a wider variety of user interactions. We *could* keep adding more tools and complex instructions to our single weather agent, but this can quickly become unmanageable and less efficient.

A more robust approach is to build an **Agent Team**. This involves:

1. Creating multiple, **specialized agents**, each designed for a specific capability (e.g., one for weather, one for greetings, one for calculations).  
2. Designating a **root agent** (or orchestrator) that receives the initial user request.  
3. Enabling the root agent to **delegate** the request to the most appropriate specialized sub-agent based on the user's intent.

**Why build an Agent Team?**

* **Modularity:** Easier to develop, test, and maintain individual agents.  
* **Specialization:** Each agent can be fine-tuned (instructions, model choice) for its specific task.  
* **Scalability:** Simpler to add new capabilities by adding new agents.  
* **Efficiency:** Allows using potentially simpler/cheaper models for simpler tasks (like greetings).

---

**1\. Define Tools for Sub-Agents**

First, let's create the simple Python functions that will serve as tools for our new specialist agents. Remember, clear docstrings are vital for the agents that will use them.

### Tools for CSV Retrieval

In [152]:
def csv_files_vector_search_tool(query: str, k: int = 5):
    """
    Perform a vector similarity search on safety procedures.

    Args:
        query (str): The search query string.
        k (int, optional): Number of top results to return. Defaults to 5.

    Returns:
        list: List of tuples (Document, score), where Document is a record
              and score is the similarity score (lower is more similar).

    Note:
        Uses the global vector_store_csv_files for the search.
    """

    vector_search_results = vector_store_csv_files.similarity_search_with_score(
        query=query, k=k
    )
    return vector_search_results

In [153]:
def hybrid_search_tool(query: str):
    """
    Perform a hybrid (vector + full-text) search on safety procedures.

    Args:
        query (str): The search query string.

    Returns:
        list: Relevant safety procedure documents from hybrid search.

    Note:
        Uses both vector_store_csv_files and record_text_search_index.
    """

    hybrid_search = MongoDBAtlasHybridSearchRetriever(
        vectorstore=vector_store_csv_files,
        search_index_name="record_text_search_index",
        top_k=5,
    )

    hybrid_search_result = hybrid_search.get_relevant_documents(query)

    return hybrid_search_result

In [154]:
from typing import Dict, Any, Optional, List
from pydantic import BaseModel, Field
from datetime import datetime

## Generalized Record Document Creator
class GenericRecord(BaseModel):
    # Flexible for any CSV columns
    data: Dict[str, Any]
    combined_info: Optional[str] = None
    embedding: Optional[List[float]] = None
    dataset_id: Optional[str] = None
    created_at: datetime = Field(default_factory=datetime.now)

    class Config:
        arbitrary_types_allowed = True


In [155]:
from typing import List

from pydantic import BaseModel, Field


## Function to Create the Document
def create_generic_record_document(row: Dict[str, Any], dataset_id: str) -> dict:
    """
    Create a standardized document for any CSV record with flexible columns.

    Args:
        row (Dict[str, Any]): The actual CSV row as a dictionary.
        dataset_id (str): Reference to the parent dataset document.

    Returns:
        dict: Cleaned and standardized MongoDB document.
    """
    try:
        # Clean keys: remove leading/trailing whitespace, lowercase, etc. (optional)
        cleaned_row = {k.strip(): v for k, v in row.items()}

        # Create combined_info string for text search
        combined_info = " ".join(f"{k}: {v}" for k, v in cleaned_row.items())

        # Package into a generic Pydantic record
        record = GenericRecord(
            data=cleaned_row,
            combined_info=combined_info,
            dataset_id=dataset_id
        )

        return record.dict()
    except Exception as e:
        raise ValueError(f"Invalid record data: {e!s}")


In [156]:
# Tool to add new record
def create_new_record(new_record: Dict[str, any]) -> dict:
    """
    Create and validate a new generic CSV record.

    Args:
        new_record (dict): Dictionary containing a row from a CSV file. 
                           Must include 'dataset_id' as a key.

    Returns:
        dict: Validated and formatted record document.

    Raises:
        ValueError: If the input data is invalid or incomplete.

    Note:
        Uses Pydantic for data validation via create_generic_record_document function.
    """
    try:
        dataset_id = new_record.pop("dataset_id", None)
        if not dataset_id:
            raise ValueError("Missing required field 'dataset_id' in the new record.")

        document = create_generic_record_document(new_record, dataset_id)
        return document

    except Exception as e:
        raise ValueError(f"Error creating new record: {e}")

In [157]:
records_embeddings_collection_tools = [
    csv_files_vector_search_tool,
    #hybrid_search_tool,
    create_new_record,
]


### Tools for Article Retrieval

In [158]:
import os
from retriever.rag_retriever import vector_search

In [159]:
DB_NAME = "diabetes_data"
COLLECTION_NAME = "docs_multimodal"
VS_INDEX_NAME = "multimodal_vector_index"
# Connect to the MongoDB collection
collection = mongodb_client[DB_NAME][COLLECTION_NAME]

In [160]:
def article_page_vector_search_tool(
    query: str,
    model: str = "sbert",
    collection_name: str = COLLECTION_NAME,
    )-> str:
    """
    Search academic papers (page-level) using a text query via multimodal embeddings.
    Returns the top 5 page-level summaries with citations.

    Args:
        query (str): The textual search query.
        model (str): Embedding model ("sbert", "clip", "voyage").
        collection (str): MongoDB collection name to search in.

    Returns:
        str: Top 5 matching pages, each with citation and summary.
        
    """
    collection_ref = mongodb_client[DB_NAME][collection_name]
    results = vector_search(query, model=model, collection=collection_ref, display_images=False)
    if not results:
        return "No relevant pages found."
    
    summaries = []
    for r in results[:5]:
        title = r.get("pdf_title", "Unknown Title")
        page = r.get("page_number", "?")
        text = r.get("summary") or r.get("page_text", "")[:300]
        summaries.append(f"[{title}, Page {page}]: {text.strip()}")

    return "\n\n".join(summaries)  # return top 5 summaries


In [161]:
import google.generativeai as genai
from PIL import Image
import requests
from io import BytesIO

model = genai.GenerativeModel("gemini-2.0-flash") #gemini-pro-vision

def extract_info_from_page_image(image: Image.Image, pdf_title: str, page_number: int) -> str:
    """
    Use Gemini to extract citations, figures/tables, and a summary from a page image.
    """
    #response = requests.get(gcs_url)
    #image = Image.open(BytesIO(response.content))

    prompt = f"""
    You are analyzing a scientific paper page image from the PDF titled: "{pdf_title}", page {page_number}.
    Please extract the following:
    1. All citations on the page (e.g., [1], Smith et al. 2020).
    2. Any tables or figures, including titles or captions.
    3. A brief summary of the page content.
    
    Return output in markdown with the following format:
    **Citations:** ...
    **Figures/Tables:** ...
    **Summary:** ...
    """

    try:
        response = model.generate_content([prompt, image])
        return response.text
    except Exception as e:
        return f"Gemini processing error: {e}"


In [162]:
GCS_PROJECT = os.getenv("GCS_PROJECT")
GCS_BUCKET = os.getenv("GCS_BUCKET")

In [163]:
from google.cloud import storage

In [164]:
# Instantiate the GCS client and bucket
gcs_client = storage.Client(project=GCS_PROJECT)
gcs_bucket = gcs_client.bucket(GCS_BUCKET)

In [165]:
from io import BytesIO
from google.cloud import storage

def get_image_from_gcs(gcs_bucket, key: str) -> bytes:
    """
    Download image bytes from GCS.

    Args:
        gcs_bucket: GCS bucket instance.
        key (str): Blob key in the bucket.

    Returns:
        bytes: Image bytes.
    """
    blob = gcs_bucket.blob(key)
    return blob.download_as_bytes()

In [166]:
from typing import Any, Dict

from langchain.agents import tool
#from pymongo import MongoClient
from PIL import Image
from io import BytesIO

def vector_search_image_tool(
    collection_name: str,
    image_bytes: Optional[bytes] = None,
    text_query: Optional[str] = None,  
    ) -> str:
    """
    Search academic papers using either an image or text query. If text is provided, it generates
    a CLIP embedding from the query and searches against page images. If image is provided, it
    searches using image embeddings.

    Args:
        image_bytes (bytes, optional): Query image content.
        text_query (str, optional): Text query to generate image embedding.
        collection_name (str): MongoDB collection name.

    Returns:
        str: Top 5 matching pages with citation and Gemini summary.
        
    """

    collection_ref = db[collection_name]

    # Determine search mode
    if image_bytes:
        try:
            query_image = Image.open(BytesIO(image_bytes))
        except Exception as e:
            return f"Invalid image input: {e}"
        results = vector_search(query_image, model="clip_image", collection=collection_ref, display_images=False)

    elif text_query:
        results = vector_search(text_query, model="clip", collection=collection_ref, display_images=False)

    else:
        return "Either image_bytes or text_query must be provided."
    
    if not results:
        return "No matching results found."
        
    output = []
    for r in results[:5]:
        pdf_title = r.get("pdf_title", "Unknown Title")
        page_number = r.get("page_number", -1)
        gcs_key = r.get("gcs_key", "")
        doc_id = r.get("_id")  # Ensure this is present in your search result

        # Check for existing gemini_summary in DB
        cached_doc = collection_ref.find_one({"_id": doc_id}, {"gemini_summary": 1})
        if cached_doc and cached_doc.get("gemini_summary"):
            summary = cached_doc["gemini_summary"]
        else:
            # If not cached, fetch image + generate Gemini summary
            try:
                page_bytes = get_image_from_gcs(gcs_bucket, gcs_key)
                page_image = Image.open(BytesIO(page_bytes))
            except Exception as e:
                return f"Failed to fetch page image: {e}"
            else:
                summary = extract_info_from_page_image(page_image, pdf_title, page_number)

                # Save summary to DB
                collection_ref.update_one(
                    {"_id": doc_id},
                    {"$set": {"gemini_summary": summary}},
                )

        citation = f"### {pdf_title}, Page {page_number}"
        output.append(f"{citation}\n{summary.strip()}")
        
    return "\n\n".join(output)


In [167]:
docs_multimodal_collection_tools = [article_page_vector_search_tool, vector_search_image_tool]

In [201]:
agent_purpose = """
You are an intelligent Multimodal Dataset and Document Assistant Agent. You help users interact with structured CSV datasets and unstructured academic articles, especially those in PDF format. These data sources are pre-indexed with vector embeddings and metadata, allowing efficient search and retrieval across both tabular and visual/textual domains.
Unless specified use all datasets to find the results. 
Your key responsibilities include:

Use csv_files_vector_search_agent and create_new_record_agent for CSV related queries.
Use article_page_vector_search_agent and vector_search_image_agent for PDF data extraction. 

1. csv_files_vector_search_agent: Searching and retrieving from CSV datasets:
  - Use search and similarity tools to find relevant rows based on natural language queries.
  - Retrieve records with the highest hybrid or semantic similarity scores.
  - Present relevant CSV content clearly, along with metadata such as column names and file name.

2. create_new_record_agent: Creating new CSV records:
  - When provided with a dictionary representing a row, use the `create_new_record` tool to validate and structure it.
  - Ensure the new record includes a `dataset_id` and other available fields.
  - Construct a `combined_info` string to support downstream embedding and search.

3. article_page_vector_search_agent: Handling academic PDF articles:
  - Use `article_page_vector_search_tool` to retrieve relevant page-level summaries from academic PDFs using a query.
  


4. csv_files_vector_search_agent: Interpreting and comparing records:
  - Explain the meaning of individual records or fields, in both CSV and PDF contexts.
  - Detect patterns across results, such as common entities, table structures, or outliers.
  - Help users compare information from CSV datasets and PDF documents when relevant.

5. csv_files_vector_search_agent: Dataset/document context awareness:
  - Understand that CSVs have varying schemas; don't assume fixed columns.
  - Use metadata like `file_name`, `n_columns`, `n_rows`, or `pdf_title`, `page_number`, `gcs_key`, and `url` to ground your responses.
  - Avoid making assumptions about missing content—only respond using indexed or retrieved information.

6. Embedding-based insights:
  - When applicable, use similarity scores or semantic reasoning to explain result relevance.
  - Explain whether a page/image was returned due to textual match, visual similarity, or both.

7. vector_search_image_agent:
    - Use `vector_search_image_tool` to perform image-based search on PDF pages using CLIP embeddings.
    - Present results with clear summaries, proper citations (including PDF title and page number), and structured insight from both page text and image content.

8. Providing structured output:
  For CSV:
    CSV Record Summary:
    - Dataset: [file_name] (ID: [dataset_id])
    - Columns: [col1, col2, ...]
    - Record:
        col1: value1
        col2: value2
        ...
    - Notes: [Any patterns, anomalies, or insights]

  For PDF:
    Article Page Match:
    - Title: [pdf_title]
    - Page: [page_number]
    - Summary: [Gemini or page_text extract]
    - Notes: [Mention if image, table, or citation was detected]

8. Acting responsibly:
  - DO NOT make up any values.
  - Clearly state when a record, page, or result is not found.
  - Always cite retrieved content from PDFs with accurate source info (title, page).
  - Respect dataset diversity and content type (CSV vs. image vs. article page).

This assistant supports analysts, researchers, and non-technical users in exploring multimodal data and extracting reliable insights from both structured CSVs and academic PDFs, using state-of-the-art embeddings and Gemini-based summarization.

DO NOT MAKE UP ANY INFORMATION.
"""


---

**2\. Define the Sub-Agents **

Now, create the `Agent` instances for our specialists. Notice their highly focused `instruction` and, critically, their clear `description`. The `description` is the primary information the *root agent* uses to decide *when* to delegate to these sub-agents.

**Best Practice:** Sub-agent `description` fields should accurately and concisely summarize their specific capability. This is crucial for effective automatic delegation.

**Best Practice:** Sub-agent `instruction` fields should be tailored to their limited scope, telling them exactly what to do and *what not* to do (e.g., "Your *only* task is...").

In [202]:
AGENT_MODEL = MODEL_GEMINI_2_0_FLASH 

In [203]:

# Create the agent
csv_files_vector_search_agent = Agent(
    name="csv_files_vector_search_agent",
    model=AGENT_MODEL,
    description="Performs a vector similarity search on safety procedures from CSV files.",
    instruction="You are an agent that performs a semantic similarity search on patient data about diabetes"
                "stored in CSV files using the 'csv_files_vector_search_tool'. "
                "Return the most relevant entries from the database based on the user's query.",
    tools=[csv_files_vector_search_tool],
)

print(f"Agent '{csv_files_vector_search_agent.name}' created using model '{AGENT_MODEL}'.")

Agent 'csv_files_vector_search_agent' created using model 'gemini-2.0-flash'.


In [204]:
hybrid_search_agent = Agent(
    name="hybrid_search_agent",
    model=AGENT_MODEL,
    description="Performs a hybrid (vector + full-text) search on CSV safety records.",
    instruction="You are an agent that retrieves the most relevant CSV records related to safety using both full-text and vector similarity search. Use the 'hybrid_search_tool'.",
    tools=[hybrid_search_tool],
)


In [205]:
create_record_agent = Agent(
    name="create_record_agent",
    model=AGENT_MODEL,
    description="Creates and validates new records for a CSV dataset.",
    instruction="You are a data creation agent. When a user submits a new CSV row with a dataset ID, validate and format it using the 'create_new_record' tool.",
    tools=[create_new_record],
)


In [206]:
article_page_search_agent = Agent(
    name="article_page_search_agent",
    model=AGENT_MODEL,
    description="Search academic papers (page-level) using a text query via multimodal embeddings.Returns the top 5 page-level summaries with citations.",
    instruction="You are an academic search agent. Use the 'article_page_vector_search_tool' to find relevant page-level results from academic papers. ",
    tools=[article_page_vector_search_tool],
)


In [207]:
vector_search_image_agent = Agent(
    name="vector_search_image_agent",
    model=AGENT_MODEL,
    description="Search academic papers using either an image or text query. If text is provided, it generates a CLIP embedding from the query and searches against page images. If image is provided, it searches using image embeddings.",
    instruction="You are a multimodal search agent. If the user provides an image or a descriptive text, use 'vector_search_image_tool' to return the top 5 academic page results with citations and summaries.",
    tools=[vector_search_image_tool],
)


In [208]:
from google.adk.agents import LlmAgent, BaseAgent
from google.adk.tools import agent_tool
from pydantic import BaseModel
#from google.adk import types
from google.adk.events import Event

class ImageGeneratorAgent(BaseAgent):
    name: str = "ImageGen"
    description: str = "Generates an image based on a prompt."

    async def _run_async_impl(self, ctx):
        prompt = ctx.session.state.get("image_prompt", "default prompt")
        image_bytes = b"..."  # replace with actual logic
        yield Event(
            author=self.name,
            content=types.Content(parts=[types.Part.from_bytes(image_bytes, "image/png")])
        )

# Wrap the custom agent as a tool
image_tool = agent_tool.AgentTool(agent=ImageGeneratorAgent())

# Use the tool in an LlmAgent
artist_agent = LlmAgent(
    name="Artist",
    model="gemini-2.0-flash",
    instruction="Create a prompt and use the ImageGen tool to generate the image.",
    tools=[image_tool]
)


---

**3\. Define the Root Agent (Medical Research Agent for Diabetes v2) with Sub-Agents**

Now, we upgrade our `search_agent`. The key changes are:

* Adding the `sub_agents` parameter: We pass a list containing the `_agent` and `_agent` instances we just created.  
* Updating the `instruction`: We explicitly tell the root agent *about* its sub-agents and *when* it should delegate tasks to them.

**Key Concept: Automatic Delegation (Auto Flow)** By providing the `sub_agents` list, ADK enables automatic delegation. When the root agent receives a user query, its LLM considers not only its own instructions and tools but also the `description` of each sub-agent. If the LLM determines that a query aligns better with a sub-agent's described capability (e.g., "Handles simple greetings"), it will automatically generate a special internal action to *transfer control* to that sub-agent for that turn. The sub-agent then processes the query using its own model, instructions, and tools.

**Best Practice:** Ensure the root agent's instructions clearly guide its delegation decisions. Mention the sub-agents by name and describe the conditions under which delegation should occur.

In [209]:
from google.adk.agents import LlmAgent
from google.adk.tools import agent_tool

# Define the purpose/instruction of the coordinator agent
COORDINATOR_PURPOSE = agent_purpose
"""
You are the coordinator agent of the search4cure.AI Diabetes Research Assistant System.

You orchestrate a team of specialized sub-agents and tools that interact with structured CSV datasets and unstructured academic documents (PDFs), both pre-embedded for semantic and visual retrieval. Your role is to:

1. Understand the user’s query in context.
2. Decide which sub-agent or tool to invoke:
    - Use `csv_files_vector_search_agent` for structured tabular data.
    - Use `article_page_vector_search_tool` to retrieve semantic matches from academic PDFs, using sbert.
    - Use `vector_search_image_tool` to retrieve visually similar content using CLIP.
    - Use `create_new_record` to add new structured entries to indexed CSV datasets.
    - Use 'artist agent' to generate image based on a query.
3. Aggregate results across modalities when necessary.
4. Format structured outputs for clarity and citation.

You ensure answers are factual, properly cited, and clearly distinguish between text, table, and image-based evidence. Use source metadata like dataset_id, file_name, pdf_title, and page_number to support your response.

DO NOT fabricate information. Always cite sources and explain reasoning clearly.
"""


# Mid-level agent combining tools
research_assistant = LlmAgent(
    name="research_assistant_for_the_cure_Diabetes",
    model="gemini-2.0-flash",
    description= COORDINATOR_PURPOSE,
    tools=[agent_tool.AgentTool(agent=csv_files_vector_search_agent), 
           agent_tool.AgentTool(agent=create_record_agent), 
           agent_tool.AgentTool(agent=article_page_search_agent), 
           agent_tool.AgentTool(agent=vector_search_image_agent),
           agent_tool.AgentTool(agent=artist_agent)]
)

# High-level agent delegating research
report_writer = LlmAgent(
    name="ReportWriter",
    model="gemini-2.0-flash",
    instruction="Write a report on the given question. Use the ResearchAssistant to gather information.",
    tools=[agent_tool.AgentTool(agent=research_assistant)]
)

---

**4\. Interact with the Agent**

We need a way to send messages to our agent and receive its responses. Since LLM calls and tool executions can take time, ADK's `Runner` operates asynchronously.

We'll define an `async` helper function (`call_agent_async`) that:

1. Takes a user query string.  
2. Packages it into the ADK `Content` format.  
3. Calls `runner.run_async`, providing the user/session context and the new message.  
4. Iterates through the **Events** yielded by the runner. Events represent steps in the agent's execution (e.g., tool call requested, tool result received, intermediate LLM thought, final response).  
5. Identifies and prints the **final response** event using `event.is_final_response()`.

**Why `async`?** Interactions with LLMs and potentially tools (like external APIs) are I/O-bound operations. Using `asyncio` allows the program to handle these operations efficiently without blocking execution.

#### Define Agent Interaction Function

In [210]:
# @title Define Agent Interaction Function

from google.genai import types # For creating message Content/Parts

async def call_agent_async(query: str, runner, user_id, session_id):
  """Sends a query to the agent and prints the final response."""
  print(f"\n>>> User Query: {query}")

  # Prepare the user's message in ADK format
  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "Agent did not produce a final response." # Default

  # Key Concept: run_async executes the agent logic and yields Events.
  # We iterate through events to find the final answer.
  async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
      # You can uncomment the line below to see *all* events during execution
      print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

      # Key Concept: is_final_response() marks the concluding message for the turn.
      if event.is_final_response():
          if event.content and event.content.parts:
             # Assuming text response in the first part
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: # Handle potential errors/escalations
             final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
          # Add more checks here if needed (e.g., specific error codes)
          break # Stop processing events once the final response is found

  print(f"<<< Agent Response: {final_response_text}")

---

**3\. Setup Runner and Session Service**

To manage conversations and execute the agent, we need two more components:

* `SessionService`: Responsible for managing conversation history and state for different users and sessions. The `InMemorySessionService` is a simple implementation that stores everything in memory, suitable for testing and simple applications. It keeps track of the messages exchanged. We'll explore state persistence more in Step 4\.  
* `Runner`: The engine that orchestrates the interaction flow. It takes user input, routes it to the appropriate agent, manages calls to the LLM and tools based on the agent's logic, handles session updates via the `SessionService`, and yields events representing the progress of the interaction.

---

**4\. Interact with the Agent Team**

Now that we've defined our root agent (`weather_agent_team` - *Note: Ensure this variable name matches the one defined in the previous code block, likely `# @title Define the Root Agent with Sub-Agents`, which might have named it `root_agent`*) with its specialized sub-agents, let's test the delegation mechanism.

The following code block will:

1.  Define an `async` function `run_team_conversation`.
2.  Inside this function, create a *new, dedicated* `InMemorySessionService` and a specific session (`session_001_agent_team`) just for this test run. This isolates the conversation history for testing the team dynamics.
3.  Create a `Runner` (`runner_agent_team`) configured to use our `weather_agent_team` (the root agent) and the dedicated session service.
4.  Use our updated `call_agent_async` function to send different types of queries (greeting, weather request, farewell) to the `runner_agent_team`. We explicitly pass the runner, user ID, and session ID for this specific test.
5.  Immediately execute the `run_team_conversation` function.

We expect the following flow:

1.  The "Hello there!" query goes to `runner_agent_team`.
2.  The root agent (`weather_agent_team`) receives it and, based on its instructions and the `greeting_agent`'s description, delegates the task.
3.  `greeting_agent` handles the query, calls its `say_hello` tool, and generates the response.
4.  The "What is the weather in New York?" query is *not* delegated and is handled directly by the root agent using its `get_weather` tool.
5.  The "Thanks, bye!" query is delegated to the `farewell_agent`, which uses its `say_goodbye` tool.



In [211]:
# @title Interact with the Search4Cure Diabetes Agent Team
import asyncio
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

# Set your root agent name
root_agent_var_name = 'research_assistant'  # Updated for your ReportWriter agent

# Check if the root agent exists
if root_agent_var_name not in globals():
    print(f"⚠️ Root agent '{root_agent_var_name}' not found. Cannot define run_team_conversation.")
    report_writer = None  # Just for safe fallback

if root_agent_var_name in globals() and globals()[root_agent_var_name]:
    async def run_team_conversation():
        print("\n--- Running Search4Cure Diabetes Research Agent Team ---")

        # Initialize in-memory session
        session_service = InMemorySessionService()
        APP_NAME = "search4cure_diabetes_team"
        USER_ID = "user_1"
        SESSION_ID = "session_diabetes_001"
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID
        )
        print(f"✅ Session initialized: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

        actual_root_agent = globals()[root_agent_var_name]

        # Initialize runner
        runner = Runner(
            agent=actual_root_agent,
            app_name=APP_NAME,
            session_service=session_service
        )
        print(f"✅ Runner created for root agent '{actual_root_agent.name}'")

        # --- Test Interactions ---
        await call_agent_async(query="Find patients over 65 with high blood pressure from the dataset.",
                               runner=runner, user_id=USER_ID, session_id=SESSION_ID)

        await call_agent_async(query="Give me the information about the historical improvements on the cure of Diabetes?",
                               runner=runner, user_id=USER_ID, session_id=SESSION_ID)

        await call_agent_async(query="Add this row to the diabetes dataset: age: 50, bmi: 29.2, glucose: 130, outcome: 1.",
                               runner=runner, user_id=USER_ID, session_id=SESSION_ID)

        await call_agent_async(query="What does the glucose column represent in the diabetes file?",
                               runner=runner, user_id=USER_ID, session_id=SESSION_ID)
        
        await call_agent_async(query="Is there a correlation between the glucose levels in the CSV and findings in any PDF pages?",
                               runner=runner, user_id=USER_ID, session_id=SESSION_ID)
        
        await call_agent_async(query="Get me a list of research papers on the topic Machine Learning Methods on the treatment of Diabetes.",
                               runner=runner, user_id=USER_ID, session_id=SESSION_ID)
        
        await call_agent_async(query="Summarize the first page of the PDF titled 'Early_Stage_Diabetes_Prediction_via_Extreme_Learning_Machine'.",
                               runner=runner, user_id=USER_ID, session_id=SESSION_ID)

    # Notebook execution
    print("ℹ️ Attempting notebook-style execution using 'await'...")
    await run_team_conversation()

    # For .py script execution (uncomment if needed):
    """
    if __name__ == "__main__":
        print("ℹ️ Running async agent team flow using asyncio.run()...")
        asyncio.run(run_team_conversation())
    """
else:
    print("⚠️ Skipping execution: root agent not defined correctly.")


ℹ️ Attempting notebook-style execution using 'await'...

--- Running Search4Cure Diabetes Research Agent Team ---
✅ Session initialized: App='search4cure_diabetes_team', User='user_1', Session='session_diabetes_001'
✅ Runner created for root agent 'research_assistant_for_the_cure_Diabetes'

>>> User Query: Find patients over 65 with high blood pressure from the dataset.
  [Event] Author: research_assistant_for_the_cure_Diabetes, Type: Event, Final: False, Content: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=FunctionCall(id='adk-9491a1d7-525e-4e63-a30b-98bb1facddd7', args={'request': 'patients over 65 with high blood pressure'}, name='csv_files_vector_search_agent'), function_response=None, text=None)] role='model'
  [Event] Author: research_assistant_for_the_cure_Diabetes, Type: Event, Final: False, Content: parts=[Part(video_metadata=None, thought=None, inline_d